In [128]:
# libaries
import random
import numpy as np
import matplotlib.pyplot as plt
import time

# character length of genome
genome_size = 15
# character in DNA
DNA_characters = 'ACGT'
# distribution of each DNA character
distribution = 0.20
# number of DNA characters
element_number = 4
# probaility out of 100 for mutation
mutation_probability = 5
# number of individuals in a population
pop_size = 100

elitism_percentage = 5

In [129]:
# individual genome for the population
class Individual:
  # initialization
  def __init__(self):
    # fitness score for genome
    self.fitness = 0
    # character string for active genome
    self.genome_active = []
    # count of A in active genome
    self.A_active = 0
    # count of A in active genome
    self.C_active = 0
    # count of G in active genome
    self.G_active = 0
    # count of T in active genome
    self.T_active = 0
    # create genome
    for i in range(0, genome_size):
      # create active genome
      self.genome_active.append(random.choice(DNA_characters))
    # update new fitness score for genome
    self.calculate_fitness()


  def calculate_fitness(self):

    # ratio used for each type of the 3 fitness scores
    even_ratio = 1/3
    alpha1_ratio = 1/3
    alpha2_ratio = 1/3

    # fitness score for evenly distributed
    even_score = self.even_distributed_fitness(distribution,
                                               genome_size,
                                               element_number)

    # fitness score for alphabetizing order by value proceeding character
    alpha1_score = self.alphabetize_1_fitness()

    # fitness score for alphabetizing correct position comparison
    alpha2_score = self.alphabetize_2_fitness()

    # calculate fitness with 3  fitness metrics
    self.fitness = (even_ratio*even_score) + (alpha1_ratio*alpha1_score) + (alpha2_ratio*alpha2_score)

  def alphabetize_1_fitness(self):
    # number of unalphabetize pairs
    misordered_pairs = 0
    # highest score, if string is alphabetized
    score = len(self.genome_active)-1
    # check all adjacent pairs in string
    for i in range(genome_size - 1):
      # count alphabetized pair
      if self.genome_active[i] > self.genome_active[i + 1]:
          misordered_pairs += 1
    # normalized fitness score
    return (score - misordered_pairs)/(score)

  def alphabetize_2_fitness(self):
    # fitness score
    score = 0
    # check if A is in correct placement
    for i in range(0, self.A_active):
      # 1 point for each A
      if self.genome_active[i] == 'A':
        score += 1
    # check if C is in correct placement
    for i in range(self.A_active, self.A_active + self.C_active):
      # 1 point for each C
      if self.genome_active[i] == 'C':
        score += 1
    # check if G is in correct placement
    for i in range(self.A_active + self.C_active, self.A_active + self.C_active + self.G_active):
      # 1 point for each G
      if self.genome_active[i] == 'G':
        score += 1
    # check if T is in correct placement
    for i in range((self.A_active + self.C_active + self.G_active), (self.A_active + self.C_active + self.G_active + self.T_active)):
      if self.genome_active[i] == 'T':
        # 1 point for each T
        score += 1
    # normalize fitness score
    norm_score = (score/(self.A_active + self.C_active + self.G_active + self.T_active))
    return norm_score

  def even_distributed_fitness(self,
                               dis_percentage,
                               string_size,
                               element_number):
    # starting penalty out of distribution percentage
    penalty = 0
    # reset count
    self.A_active = 0
    self.C_active = 0
    self.G_active = 0
    self.T_active = 0
    # number of characters in distribution range
    upper_limit = dis_percentage * string_size
    # highest possible penalty
    max_penalty = (element_number - 1) * (string_size * dis_percentage)
    # count elements in string
    for c in self.genome_active:
      if c == 'G':
        self.G_active += 1
      elif c == 'T':
        self.T_active += 1
      elif c == 'C':
        self.C_active += 1
      elif c == 'A':
        self.A_active += 1
    # number of characters out of distribution percentage
    penalty_A = (self.A_active - upper_limit)
    penalty_C = (self.C_active - upper_limit)
    penalty_G = (self.G_active - upper_limit)
    penalty_T = (self.T_active - upper_limit)
    # penalize 1 point for every A under distribution percentage
    if penalty_A < 0:
      penalty = penalty - abs(penalty_A)
    # penalize 1 point for every C under distribution percentage
    if penalty_C < 0:
      penalty = penalty -abs(penalty_C)
    # penalize 1 point for every G under distribution percentage
    if penalty_G < 0:
      penalty = penalty - abs(penalty_G)
    # penalize 1 point for every T under distribution percentage
    if penalty_T < 0:
      penalty = penalty - abs(penalty_T)
    # raw fitness score
    score = max_penalty + penalty
    # normalize fitness score
    norm_score = score/max_penalty
    return norm_score

  def mutation(self):
    for i in range(0, genome_size):
      if(random.uniform(0, 100) < mutation_probability):
          self.genome_active[i] = random.choice(DNA_characters)
    # update new fitness score for genome
    self.calculate_fitness()

  def print(self):
    # print("\nFinal Genome")
    print("\nActive Genome: " + str(self.genome_active) + "Fitness: " + str(self.fitness))
    #print("Hidden Genome: " + str(self.genome_hidden))
    #print("Fitness: " + str(self.fitness))

  def copy(self, source):
    self.fitness = source.fitness
    for i in range(0, genome_size):
      self.genome_active[i] = source.genome_active[i]
    self.A_active = source.A_active
    self.C_active = source.C_active
    self.G_active = source.G_active
    self.T_active = source.T_active

  def __str__(self):
    output = ""
    output += "Active Genome: " + str(self.genome_active) + "\n"
    # output += "Hidden Genome: " + str(self.genome_hidden) + "\n"
    output += "Fitness: " + str(self.fitness) + "\n\n"
    return output


  # overwrite less than operator for individuals
  def __lt(self, other):
      return self.fitness < other.fitness

In [130]:
class DiploidIndiv(Individual):
  # initialization
  def __init__(self):
    super().__init__()
    # character string for hidden genome
    self.genome_hidden = []
    for i in range(0, genome_size):
      # create active genome
      self.genome_hidden.append(random.choice(DNA_characters))
    # no need to calc fitness again because hidden doesn't change fitness

  def mutation(self):
    for i in range(0, genome_size):
      if(random.uniform(0, 100) < mutation_probability):
          self.genome_active[i] = random.choice(DNA_characters)

          # We should decide whether or not we want to mutate the hidden genome
          self.genome_hidden[i] = random.choice(DNA_characters)

    # update new fitness score for genome
    self.calculate_fitness()



  def print(self):
    # print("\nFinal Genome")
    print("\nActive Genome: " + str(self.genome_active) + "Fitness: " + str(self.fitness))
    print("Hidden Genome: " + str(self.genome_hidden))
    #print("Fitness: " + str(self.fitness))

  def copy(self, source):
    self.fitness = source.fitness
    for i in range(0, genome_size):
      self.genome_active[i] = source.genome_active[i]
      self.genome_hidden[i] = source.genome_hidden[i]
    self.A_active = source.A_active
    self.C_active = source.C_active
    self.G_active = source.G_active
    self.T_active = source.T_active

  def __str__(self):
    output = ""
    output += "Active Genome: " + str(self.genome_active) + "\n"
    output += "Hidden Genome: " + str(self.genome_hidden) + "\n"
    output += "Fitness: " + str(self.fitness) + "\n\n"
    return output

# Overwrite less-than operator for individuals
def __lt__(self, other):
    self.calculate_fitness()  # Ensure fitness is updated
    return self.fitness < other.fitness


In [131]:
i1=Individual()
print(i1)

i2=DiploidIndiv()
print(i2)

Active Genome: ['C', 'C', 'A', 'A', 'T', 'C', 'C', 'C', 'A', 'A', 'C', 'T', 'T', 'A', 'T']
Fitness: 0.6603174603174603


Active Genome: ['T', 'C', 'T', 'G', 'C', 'T', 'G', 'A', 'C', 'T', 'C', 'C', 'G', 'C', 'C']
Hidden Genome: ['G', 'C', 'G', 'A', 'T', 'T', 'A', 'C', 'A', 'A', 'C', 'T', 'C', 'T', 'T']
Fitness: 0.4703703703703704




In [132]:
# population classs
class population:
  # initialization
  def __init__(self,hap_or_dip):
    self.average_fitness = 0
    self.the_pop = []
    self.haploid=True
    # pop of Haploids
    if(hap_or_dip==1):
      for i in range(0,pop_size):
        self.the_pop.append(Individual())
    # pop  of Diploids
    else:
      self.haploid=False
      for i in range(0,pop_size):
        self.the_pop.append(DiploidIndiv())
    self.calculate_average_fitness()

  def calculate_average_fitness(self):
    total_fitness = 0
    for i in range(0, pop_size):
      total_fitness += self.the_pop[i].fitness
    self.average_fitness = total_fitness/pop_size

  def get_best_fitness(self):
    best = self.the_pop[0].fitness

    for i in range(1, pop_size):
      if self.the_pop[i].fitness > best:
        best = self.the_pop[i].fitness

    return best

  def __str__(self):
    output = ""
    for i in range(0, pop_size):
      output += str(self.the_pop[i])
      output += "\n"
    return output

  def tournament(self):
    t = 3
    best_so_far = random.randint(0, pop_size-1)
    best_fitness = self.the_pop[best_so_far].fitness
    for i in range(0, t - 1):
      current = random.randint(0, pop_size - 1)
      current_fitness = self.the_pop[current].fitness
      if current_fitness > best_fitness:
        best_so_far = current
        best_fitness = current_fitness
    return best_so_far

  def asexual_generation(self):
    tempPop = population()

    for i in range(0, elitism_percentage):
      tempPop.the_pop[i].copy(self.the_pop[i])

    if(self.haploid):
      tempPop = population(1)
    else:
      tempPop=population(2)

    self.the_pop.sort(reverse=True)

    # create temporary population with tournament selection
    for i in range(0, pop_size):
      # select parent 1
      parent1 = self.tournament()
      # make new individual
      tempPop.the_pop[i].copy(self.the_pop[parent1])
      tempPop.the_pop[i].mutation()
    # copy newly created population to current population
    for i in range(elitism_percentage, pop_size):
      self.the_pop[i].copy(tempPop.the_pop[i])
    # calculate new population statistics
    self.calculate_average_fitness()

  def sexual_generation(self):
    if(self.haploid):
      tempPop1 = population(1)
      tempPop2 = population(1)
    else:
      tempPop1 = population(2)
      tempPop2 = population(2)

    self.the_pop.sort(reverse=True)

    # create temporary populations of parents with tournament selection
    #   Note: tempPop[0] will mate with tempPop[0]... so on to make new populatiom
    for i in range(0, pop_size):
      # select parent 1
      parent1 = self.tournament()
      # select parent 2
      parent2 = self.tournament()
      # make 2 new individuals in parallel populations
      tempPop1.the_pop[i].copy(self.the_pop[parent1])
      tempPop2.the_pop[i].copy(self.the_pop[parent2])

    for i in range(0, elitism_percentage):
          tempPop1.the_pop[i].copy(self.the_pop[i])
          tempPop2.the_pop[i].copy(self.the_pop[i])

    # sexual reproduction
    for p in range(elitism_percentage,pop_size):
      if(self.haploid):
        self.the_pop[i]=self.__make_haploid_baby(tempPop1.the_pop[p],tempPop2.the_pop[p])
      else:
        self.the_pop[i]=self.__make_diploid_baby(tempPop1.the_pop[p],tempPop2.the_pop[p])

      # Note: mutation happens in the __make_baby functions
    # calculate new population statistics
    self.calculate_average_fitness()


  # private function to set baby's genomes for haploid
  def __make_haploid_baby(self,parent_1,parent_2):
    baby = Individual()
    # for each genome, flip to see which parent passes on their gene
    for g in range(0,genome_size):
      coin_flip=random.randint(0,1)
      if(coin_flip==0):
        baby.genome_active[g]=parent_1.genome_active[g]
      else:
        baby.genome_active[g]=parent_2.genome_active[g]
    # mutate and return baby
    baby.mutation()
    return baby

  # private function to set baby's genomes for diploid
  def __make_diploid_baby(self,parent_1,parent_2):
    baby = DiploidIndiv()
    # for each genome, flip to see which parent passes on their gene
    for g in range(0,genome_size):
      # choose parent for baby's ACTIVE GENOME
      active_parent = random.randint(0,1)
      # choose if baby gets that parent's active or hidden genome as its active
      #      baby gets its ACTIVE gene from parent 1
      if(active_parent==0):
        coin_flip=random.randint(0,1)
        # baby inherits ACTIVE as active
        if(coin_flip==0):
          baby.genome_active[g]=parent_1.genome_active[g]
        # baby inherits HIDDEN as active
        else:
          baby.genome_active[g]=parent_1.genome_hidden[g]
        # flip to see which gene baby gets as hidden from parent 2
        coin_flip=random.randint(0,1)
        # baby inherits ACTIVE as hidden
        if(coin_flip==0):
          baby.genome_hidden[g]=parent_2.genome_active[g]
        # baby inherits HIDDEN as hidden
        else:
          baby.genome_hidden[g]=parent_2.genome_hidden[g]

      #       baby gets its ACTIVE gene from parent 2
      else:
        coin_flip=random.randint(0,1)
        # baby inherits ACTIVE as active
        if(coin_flip==0):
          baby.genome_active[g]=parent_2.genome_active[g]
        # baby inherits HIDDEN as active
        else:
          baby.genome_active[g]=parent_2.genome_hidden[g]
        # flip to see which gene baby gets as hidden from parent 1
        coin_flip=random.randint(0,1)
        # baby inherits ACTIVE as hidden
        if(coin_flip==0):
          baby.genome_hidden[g]=parent_1.genome_active[g]
        # baby inherits HIDDEN as hidden
        else:
          baby.genome_hidden[g]=parent_1.genome_hidden[g]

    # mutate and return baby
    baby.mutation()
    return baby


In [133]:
# p1=population(1)
# p1.calculate_average_fitness()
# print(p1)
# print(p1.average_fitness)

In [134]:
# p2=population(2)
p2=population(1)
p2.calculate_average_fitness()
# print(p2)
# print(p2.average_fitness)

In [135]:
count = 0
for i in range(0, 20000):
  percent = .97
  max_fitness = 1.0
  p2.sexual_generation()

  if p2.get_best_fitness() == max_fitness:
    break;

  p2.calculate_average_fitness()
  print("Average: ", p2.average_fitness)
  if p2.average_fitness > percent:
    print("\nfound average of: ", percent)
    break
  if i == 99999:
    print("\ndid not find average")
  count += 1

print("generation: ", count)
print("---------------------------")
#print(p1)

TypeError: '<' not supported between instances of 'Individual' and 'Individual'